In [2]:
def factorial(x):
    f = 1
    for i in range(1, x+1):
        f *= i
    return f


%time a = factorial(100000)

CPU times: total: 3.28 s
Wall time: 3.44 s


In [3]:
from numba import jit

@jit(nopython=True, fastmath=True)
def factorial(x):
    f = 1
    for i in range(1, x+1):
        f *= i
    return f


%time a = factorial(100000)

CPU times: total: 547 ms
Wall time: 642 ms


In [20]:
from numba import jit

@jit(nopython=True, fastmath=True, cache=True)
def factorial(x):
    f = 1
    for i in range(1, x+1):
        f *= i
    return f


%time a = factorial(100000)

CPU times: total: 0 ns
Wall time: 8 ms


In [33]:
from numba import jit, prange

@jit(nopython=True, fastmath=True, cache=True, parallel=True)
def factorial(x):
    f = 1
    for i in prange(1, x+1):
        f *= i
    return f


%time a = factorial(100000)

CPU times: total: 15.6 ms
Wall time: 18 ms


In [34]:
import taichi as ti

@ti.func
def factorial_taichi(x):
    f = 1
    for i in range(1, x+1):
        f *= i
    return f


ti.init()
f = ti.field(dtype=ti.i32, shape=())
@ti.kernel
def compute():
    f[None] = factorial_taichi(100000)

%time a = compute()

[Taichi] version 1.6.0, llvm 15.0.1, commit f1c6fbbd, win, python 3.11.5
[Taichi] Starting on arch=x64
CPU times: total: 109 ms
Wall time: 161 ms


In [ ]:
# визуализация пузырьковой сортировки при помощи Pygame, если не работает, вынеси в отдельный файл

import pygame as pg
import random
from time import sleep

# инициализация Pygame
pg.init()

# константы для окна и столбиков
SCREEN_WIDTH = 400  # 800
SCREEN_HEIGHT = 600
BAR_WIDTH = 6
BAR_HEIGHT_UNIT = 4
BAR_GAP = 2

# цвета
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GRAY = (128, 128, 128)
RED = (255, 0, 0)

# создание окна
screen = pg.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))

# создание случайного списка для сортировки
bars = [random.randint(1, 150) for _ in range(50)]

# функция отрисовки столбиков
def draw_bars(current_index=None, swapped_index=None):
    screen.fill(GRAY)
    for i, height in enumerate(bars):
        x = i * (BAR_WIDTH + BAR_GAP) + BAR_GAP
        y = SCREEN_HEIGHT - BAR_HEIGHT_UNIT * height
        color = RED if i == current_index or i == swapped_index else WHITE
        pg.draw.rect(screen, color, (x, y, BAR_WIDTH, BAR_HEIGHT_UNIT * height))

# функция пузырьковой сортировки
def bubble_sort():
    for i in range(len(bars)):
        for j in range(len(bars) - i - 1):
            if bars[j] > bars[j+1]:
                # обмен элементов
                bars[j], bars[j+1] = bars[j+1], bars[j]
                # отрисовка обмена
                draw_bars(j, j+1)
                sleep(0.01)  # тут мы регулируем скорость анимации
                pg.display.update()

# основной цикл программы
clock = pg.time.Clock()
running = True
while running:
    for event in pg.event.get():
        if event.type == pg.QUIT:
            running = False

    # отрисовка начального состояния
    draw_bars()
    pg.display.flip()
    fps = clock.get_fps()
    pg.display.set_caption(f"{fps = }")

    # сортировка и отрисовка
    bubble_sort()

    # ожидание закрытия окна
    while True:
        event = pg.event.wait()
        if event.type == pg.QUIT:
            running = False
            break

# завершение Pygame
pg.quit()

In [39]:
import random
numbers_ = [random.randint(0, 100) for _ in range(10_000)]
numbers = numbers_.copy()


# Функция для пузырьковой сортировки
def bubblesort(numbers):
    n = len(numbers)
    for i in range(n):
        for j in range(0, n-i-1):
            if numbers[j] > numbers[j+1]:
                numbers[j], numbers[j+1] = numbers[j+1], numbers[j]
    return numbers


def bubblesort_opt(numbers):
    n = len(numbers)
    for i in range(n, -1, -1):
        swapped = False
        for j in range(i-1):
            if numbers[j] > numbers[j + 1]:
                numbers[j], numbers[j + 1] = numbers[j + 1], numbers[j]
                swapped = True
        if not swapped:
            break
    return numbers


import numpy as np
def bubblesort_numpy(numbers):
    numbers = np.array(numbers)
    numbers = np.sort(numbers)
    return numbers.tolist()


numbers = numbers_.copy()
%timeit bubblesort(numbers)

numbers = numbers_.copy()
%timeit bubblesort_opt(numbers)

numbers = numbers_.copy()
%timeit bubblesort_numpy(numbers)

4.99 s ± 76.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
722 µs ± 27.7 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.12 ms ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Нахождение простых чисел на C 

```C 
#include <stdio.h>

int is_prime(int num) {
    if(num < 2) {
        return 0;
    }
    for(int i = 2; i <= num/2; i++) {
        if(num % i == 0) {
            return 0;
        }
    }
    return 1;
}

void primes() {
    int count = 0;
    for(int x = 2; x <= 1000000; x++) {
        if(is_prime(x)) {
            printf("%d ", x);
            count++;
            if(count == 20) {
                break;
            }
        }
    }
}

int main() {
    primes();
    return 0;
}
```

In [5]:
from numba import jit

@jit(nopython=True, cache=True)
def primes():
    primes = []
    for x in range(2, 1_000_000):
        for d in range(2, int(x**0.5)+1):
            if x % d == 0:
                break 
        else:
            primes.append(x)
    
    return primes


%timeit primes()

263 ms ± 7.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
import numpy as np


def primes():
    sieve = np.ones(1_000_000//2, dtype=bool)
    for i in range(3, int(1_000_000**0.5)+1, 2):
        if sieve[i//2]:
            sieve[i*i//2::i] = False
    primes = 2*np.nonzero(sieve)[0][1::]+1
    return primes.tolist()[:20]

%timeit primes()

4.64 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
